In [ ]:
# %% [markdown]
# # Analyse & Qualité des Données - Dataset Cancer du Sein
# - Fusion données tabulaires & images
# - Exploration & visualisation
# - Analyse de la qualité des données
# - Préparation d'un dataset nettoyé

# %%
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Optionnel : seaborn si tu veux plus de style
import seaborn as sns

plt.style.use("default")

BASE = Path("CancerSeins")   # adapter si besoin
CSV_DIR = BASE / "csv"
IMG_DIR = BASE / "jpeg"

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)


In [ ]:
# %% [markdown]
# ## 1. Chargement & fusion des données

# %%
# Charger les fichiers CSV
meta = pd.read_csv(CSV_DIR / "meta.csv")
dicom_info = pd.read_csv(CSV_DIR / "dicom_info.csv")

calc_train = pd.read_csv(CSV_DIR / "calc_case_description_train_set.csv")
calc_test  = pd.read_csv(CSV_DIR / "calc_case_description_test_set.csv")
mass_train = pd.read_csv(CSV_DIR / "mass_case_description_train_set.csv")
mass_test  = pd.read_csv(CSV_DIR / "mass_case_description_test_set.csv")

# Concat train + test pour chaque type de lésion
calc = pd.concat([calc_train, calc_test], ignore_index=True)
mass = pd.concat([mass_train, mass_test], ignore_index=True)

print("meta:", meta.shape)
print("dicom_info:", dicom_info.shape)
print("calc:", calc.shape)
print("mass:", mass.shape)

# %%
# Fusion meta + dicom_info
df = meta.merge(dicom_info, on="image_id", how="left")

# Ajouter description des calcifications & masses
df = df.merge(calc, on="image_id", how="left", suffixes=("", "_calc"))
df = df.merge(mass, on="image_id", how="left", suffixes=("", "_mass"))

print("DataFrame fusionné:", df.shape)
df.head()


In [ ]:
# %% [markdown]
# ## 2. Ajout du lien vers les images JPEG

# %%
def find_first_jpg(dicom_id):
    """Retourne le chemin du premier .jpg présent dans le dossier dicom_id."""
    folder_path = IMG_DIR / str(dicom_id)
    if folder_path.exists() and folder_path.is_dir():
        jpgs = list(folder_path.glob("*.jpg"))
        if jpgs:
            return jpgs[0].as_posix()
    return np.nan

# attention : peut prendre un peu de temps la première fois
df["image_path"] = df["dicom_id"].apply(find_first_jpg)

# Statistiques de correspondance
n_total = len(df)
n_found = df["image_path"].notna().sum()
print(f"Images trouvées : {n_found}/{n_total} ({n_found/n_total*100:.2f} %)")

df[["image_id", "dicom_id", "image_path"]].head()


In [ ]:
# %% [markdown]
# ## 3. Vue d'ensemble des données

# %%
print("Dimensions :", df.shape)

print("\nTypes des colonnes :")
print(df.dtypes)

# %%
# Aperçu aléatoire
df.sample(5, random_state=42)


In [ ]:
# %% [markdown]
# ## 4. Qualité des données : valeurs manquantes & doublons

# %%
# Fonction utilitaire pour résumé des NA
def missing_report(dataframe):
    na_count = dataframe.isna().sum()
    na_pct = na_count / len(dataframe) * 100
    rep = pd.DataFrame({
        "nb_manquants": na_count,
        "%_manquants": na_pct
    })
    rep = rep[rep["nb_manquants"] > 0].sort_values("%_manquants", ascending=False)
    return rep

missing_df = missing_report(df)
missing_df.head(20)

# %%
# Visualisation des % de manquants (top 30)
if not missing_df.empty:
    top_missing = missing_df.head(30)
    
    plt.figure(figsize=(10, 6))
    plt.barh(top_missing.index, top_missing["%_manquants"])
    plt.xlabel("% de valeurs manquantes")
    plt.title("Top 30 des variables avec valeurs manquantes")
    plt.gca().invert_yaxis()
    plt.show()

# %%
# Doublons sur image_id (clé logique)
n_duplicates = df.duplicated(subset=["image_id"]).sum()
print(f"Doublons potentiels sur image_id : {n_duplicates}")


In [ ]:
# %% [markdown]
# ## 5. Statistiques descriptives & cardinalité

# %%
# Colonnes numériques (hors identifiants)
id_like = ["image_id", "patient_id", "study_id", "dicom_id"]
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in id_like]

df[num_cols].describe().T

# %%
# Cardinalité des variables catégorielles
cat_cols = df.select_dtypes(include=["object"]).columns

card = pd.DataFrame({
    "nb_modalites": [df[c].nunique(dropna=True) for c in cat_cols],
    "nb_manquants": [df[c].isna().sum() for c in cat_cols]
}, index=cat_cols).sort_values("nb_modalites", ascending=False)

card.head(20)


In [ ]:
# %% [markdown]
# ## 6. Distribution de la variable cible

# %%
# Essaie plusieurs noms possibles selon tes colonnes
target_candidates = [c for c in df.columns if "path" in c.lower() or "malign" in c.lower() or "class" in c.lower()]
target_candidates


In [ ]:
# %% [markdown]
# ## 8. Visualisations bivariées

# %%
if TARGET_COL in df.columns:
    # Boxplots pour quelques variables numériques
    for col in num_cols[:8]:  # limite à 8 pour ne pas exploser
        plt.figure(figsize=(6,4))
        sns.boxplot(x=df[TARGET_COL], y=df[col])
        plt.title(f"{col} selon {TARGET_COL}")
        plt.tight_layout()
        plt.show()


In [ ]:
# %% [markdown]
# ## 7. Visualisations univariées (variables numériques)

# %%
for col in num_cols:
    plt.figure(figsize=(6,4))
    plt.hist(df[col].dropna(), bins=30)
    plt.title(f"Distribution de {col}")
    plt.xlabel(col)
    plt.ylabel("Effectif")
    plt.tight_layout()
    plt.show()


In [ ]:
# %% [markdown]
# ## 9. Matrice de corrélation

# %%
corr = df[num_cols].corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Matrice de corrélation (variables numériques)")
plt.tight_layout()
plt.show()


In [3]:
import pandas as pd
from pathlib import Path

BASE = Path("../..") / "CancerSeins"

# --- 1. Charger les CSV ---
meta = pd.read_csv(BASE / "csv/meta.csv")
dicom_info = pd.read_csv(BASE / "csv/dicom_info.csv")
calc_train = pd.read_csv(BASE / "csv/calc_case_description_train_set.csv")
calc_test = pd.read_csv(BASE / "csv/calc_case_description_test_set.csv")
mass_train = pd.read_csv(BASE / "csv/mass_case_description_train_set.csv")
mass_test = pd.read_csv(BASE / "csv/mass_case_description_test_set.csv")


In [4]:
print("Meta DataFrame:")
print(meta.head())


Meta DataFrame:
                                   SeriesInstanceUID  \
0  1.3.6.1.4.1.9590.100.1.2.117041576511324414842...   
1  1.3.6.1.4.1.9590.100.1.2.438738396107617880132...   
2  1.3.6.1.4.1.9590.100.1.2.767416741131676463382...   
3  1.3.6.1.4.1.9590.100.1.2.296931352612305599800...   
4  1.3.6.1.4.1.9590.100.1.2.436657670120353100077...   

                                    StudyInstanceUID Modality  \
0  1.3.6.1.4.1.9590.100.1.2.229361142710768138411...       MG   
1  1.3.6.1.4.1.9590.100.1.2.195593486612988388325...       MG   
2  1.3.6.1.4.1.9590.100.1.2.257901172612530623323...       MG   
3  1.3.6.1.4.1.9590.100.1.2.109468616710242115222...       MG   
4  1.3.6.1.4.1.9590.100.1.2.380627129513562450304...       MG   

  SeriesDescription BodyPartExamined  SeriesNumber Collection  Visibility  \
0   ROI mask images           BREAST             1  CBIS-DDSM           1   
1   ROI mask images           BREAST             1  CBIS-DDSM           1   
2   ROI mask images      

In [5]:

# Fusionner calc + mass
calc = pd.concat([calc_train, calc_test], ignore_index=True)
mass = pd.concat([mass_train, mass_test], ignore_index=True)

# --- 2. Fusionner meta + dicom ---
df = meta.merge(dicom_info, on="image_id", how="left")

# --- 3. Ajouter description des cas ---
df = df.merge(calc, on="image_id", how="left") \
       .merge(mass, on="image_id", how="left")

# --- 4. Construire le chemin vers les images ---
def get_image_path(row):
    folder = row["dicom_id"]
    folder_path = BASE / "jpeg" / folder
    if folder_path.exists():
        files = list(folder_path.glob("*.jpg"))
        if len(files) > 0:
            return str(files[0])
    return None

df["image_path"] = df.apply(get_image_path, axis=1)

print(df.head())
print(df["image_path"].notnull().mean()*100, "% images retrouvées")


KeyError: 'image_id'